## 1. Connect


In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('../data/checking-logs.sqlite')


## 2. Test group results


In [2]:
test_results = pd.io.sql.read_sql(
    """
    SELECT
        CASE
            WHEN t.first_commit_ts < t.first_view_ts THEN 'before'
            ELSE 'after'
        END AS time,
        AVG((d.deadlines - strftime('%s', t.first_commit_ts)) / 3600.0) AS avg_diff
    FROM test t
    JOIN deadlines d ON t.labname = d.labs
    WHERE t.labname != 'project1'
      AND t.uid IN (
          SELECT uid
          FROM test
          WHERE labname != 'project1'
          GROUP BY uid
          HAVING
              SUM(CASE WHEN first_commit_ts < first_view_ts THEN 1 ELSE 0 END) > 0
              AND SUM(CASE WHEN first_commit_ts > first_view_ts THEN 1 ELSE 0 END) > 0
      )
    GROUP BY time
    ORDER BY time DESC;
    """,
    conn
)
test_results


,time,avg_diff
0,before,61.156632
1,after,105.229241


## 3. Control group results


In [3]:
control_results = pd.io.sql.read_sql(
    """
    SELECT
        CASE
            WHEN c.first_commit_ts < c.first_view_ts THEN 'before'
            ELSE 'after'
        END AS time,
        AVG((d.deadlines - strftime('%s', c.first_commit_ts)) / 3600.0) AS avg_diff
    FROM control c
    JOIN deadlines d ON c.labname = d.labs
    WHERE c.labname != 'project1'
      AND c.uid IN (
          SELECT uid
          FROM control
          WHERE labname != 'project1'
          GROUP BY uid
          HAVING
              SUM(CASE WHEN first_commit_ts < first_view_ts THEN 1 ELSE 0 END) > 0
              AND SUM(CASE WHEN first_commit_ts > first_view_ts THEN 1 ELSE 0 END) > 0
      )
    GROUP BY time
    ORDER BY time DESC;
    """,
    conn
)
control_results


,time,avg_diff
0,before,99.901448
1,after,118.144571


## 4. Close


In [4]:
conn.close()


## 5. Conclusion
Answer the question: did the hypothesis turn out to be true?
